In [3]:
from osgeo import ogr
def vec_sel(path, path_out, sel_field, sel_field_value):  ##定义函数，通过sel_field和sel_field_value来选择矢量
    vector = ogr.Open(path)            ##根据路径打开矢量
    layer = vector.GetLayer(0)         ##获取图层
    
    driver = ogr.GetDriverByName('ESRI Shapefile')   ##创建数据驱动，用来写出文件
    vector_out = driver.CreateDataSource(path_out)   ##通过数据驱动，创建数据源
    layer_out = vector_out.CreateLayer('vector_out', geom_type = ogr.wkbPolygon, 
                                       srs=layer.GetSpatialRef())    ##创建图层

    fea_defn = layer_out.GetLayerDefn()    ##创建要素定义（图层定义）

    for j in range(layer.GetLayerDefn().GetFieldCount()):   ##创建循环，循环次数为原数据中的字段数，用来进行字段名称的写入
        field_defn = layer.GetLayerDefn().GetFieldDefn(j)   ##获取原数据中每一条字段的定义
        layer_out.CreateField(field_defn)                   ##根据字段定义在输出图层中写入字段的名称

    for i in range(layer.GetFeatureCount()):                ##创建循环，遍历原数据中的每一个要素，选择符合条件的写入
        fea = layer.GetFeature(i)                           ##获得原数据中的每一个要素
        
        for j in range(fea.GetFieldCount()):            ##遍历这个要素的字段，选择符合条件的字段名和字段值
            field_defn = fea.GetFieldDefnRef(j)         ##获取字段的定义，用字段的定义来获取这个字段的名称
            field_value = fea.GetField(j)               ##获取每一个字段的字段值

            ##定义写入条件：当这个要素的sel_field字段的值为sel_field_value时，才进行矢量的写入
            if str(field_defn.GetName())==sel_field and str(field_value)==sel_field_value:  
                fea_out = ogr.Feature(fea_defn)                 ##根据要素定义创建输出要素
                fea_out.SetGeometry(fea.geometry())             ##将这个符合条件的要素的几何信息写入输出要素中
                for k in range(fea.GetFieldCount()):            ##创建循环将要素的属性写入
                    field_defn = fea.GetFieldDefnRef(k)         ##获取字段的定义，用字段的定义来获取这个字段的名称
                                   
                    field_name = str(field_defn.GetName())      ##获取字段的名称

                    fea_out.SetField(field_name, fea.GetField(k))   ##通过获取到的字段的值和字段的名称将字段的值写入到字段中
                layer_out.CreateFeature(fea_out)                ##将输出要素写入到创建的图层中

    ##关闭两个文件
    vector = None
    vector_out = None

    return


In [7]:
path = 'data/Section-6/kunming_wgs84.shp' # 读入文件的路径
path_out = 'data/Section-6/selected_areas_3.shp'  # 写出文件的路径

# 定义字段
field = 'dt_name'

# 定义选择的字段
field_value = 'wuhua'

# read(path=path)
# 调用 vec_sel 函数，并传递选定的区域和字段列表
vec_sel(path=path,path_out=path_out, sel_field=field, sel_field_value=field_value)

